In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [3]:
file_name = 'sentiment_polarity/polarity_gold_lsa_topics'
extension = '.csv'
df_gold = pd.read_csv(file_name + extension, sep=',')
df_gold.head()

,id,TOPIC0,TOPIC1,TOPIC2,TOPIC3,TOPIC4,TOPIC5,TOPIC6,TOPIC7,TOPIC8,TOPIC9,TOPIC10,TOPIC11,TOPIC12,TOPIC13,TOPIC14,TOPIC15,TOPIC16,TOPIC17,TOPIC18,TOPIC19,TOPIC20,TOPIC21,TOPIC22,TOPIC23,...,TOPIC1176,TOPIC1177,TOPIC1178,TOPIC1179,TOPIC1180,TOPIC1181,TOPIC1182,TOPIC1183,TOPIC1184,TOPIC1185,TOPIC1186,TOPIC1187,TOPIC1188,TOPIC1189,TOPIC1190,TOPIC1191,TOPIC1192,TOPIC1193,TOPIC1194,TOPIC1195,TOPIC1196,TOPIC1197,TOPIC1198,TOPIC1199,class
0,4518,0.047554,-0.000308,0.023648,0.084735,0.000874,0.063103,-0.005063,-0.009235,0.045759,-0.051397,0.102817,-0.013510,0.076356,0.037979,0.055340,0.014945,0.004636,0.037343,0.023181,-0.035382,-0.015663,-0.012293,-0.030427,-0.066387,...,-0.007484,-0.004622,0.014430,0.022647,0.000322,0.007454,0.022939,0.010841,-0.015993,0.003491,0.013125,0.005661,-0.000679,-0.002385,0.002657,-0.000948,0.001994,-0.020170,-0.005735,0.002075,-0.008209,0.009179,-0.000570,0.005638,pos
1,10415,0.057894,-0.002123,0.041045,0.059079,0.003419,0.026197,0.008809,-0.001602,0.037326,-0.037092,0.014471,-0.007263,0.022827,0.000362,-0.035306,-0.028569,-0.012759,-0.048049,0.006494,-0.025306,0.004487,-0.038562,-0.012656,-0.020215,...,0.076282,0.001432,-0.027812,-0.025739,-0.022130,-0.021077,-0.019178,0.077735,0.084677,-0.022941,-0.005463,0.035403,0.055995,-0.017173,0.018196,0.015433,0.001089,-0.024572,0.094193,-0.088402,0.015108,0.007066,-0.042577,-0.058843,neg
2,7098,0.226327,0.041921,0.124921,0.291891,-0.030388,0.082626,-0.037958,-0.291264,0.184703,-0.155528,0.078627,0.249147,0.169286,-0.218392,-0.209313,-0.574267,0.040396,-1.118784,-0.225254,-0.044344,-0.337657,0.143748,-0.004862,0.179439,...,-0.017312,0.032866,-0.062766,-0.023922,-0.057547,0.089804,0.005430,-0.035278,0.028020,0.029487,-0.053324,-0.000339,0.055870,0.073867,-0.076718,0.000163,0.047404,-0.025245,-0.004108,0.027324,-0.038161,-0.021096,-0.018557,0.053247,neg
3,4396,0.011175,-0.001934,0.005822,0.014736,-0.001519,0.000643,0.010770,-0.004277,0.001010,-0.005591,0.011212,-0.006614,0.005003,-0.008331,0.001702,-0.005254,0.000745,0.000625,0.007746,0.004145,0.007582,0.011021,0.013600,-0.009401,...,-0.055257,0.002263,0.020749,-0.019761,0.014488,0.050875,0.037721,0.022402,-0.018292,-0.009705,0.048978,-0.014046,-0.001058,0.003672,-0.069564,0.025773,0.011295,0.048014,-0.022640,-0.048314,0.035679,0.012249,-0.026116,-0.004993,pos
4,2812,0.023912,0.002982,0.008951,0.038452,0.015734,0.018588,0.004741,0.005001,-0.004422,-0.014483,0.027652,0.007508,0.023304,-0.007140,-0.002079,-0.017393,-0.019381,-0.010030,0.023909,0.016616,-0.037448,-0.000804,-0.024296,0.018758,...,0.018080,-0.005512,0.020392,0.008568,-0.040549,-0.013891,0.045739,-0.077711,-0.088563,0.009808,-0.033073,0.024886,-0.069634,-0.018741,0.027686,0.057021,0.079538,-0.105252,0.011196,-0.060068,-0.029165,0.006014,-0.042851,-0.002392,pos


In [4]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

def feature_extraction(dataframe, features):    
    X_esc = dataframe.drop(columns=['class','id'])
    Y = dataframe['class']    

    #Entrenamiento del Modelo
    predictor = ExtraTreesClassifier()
    predictor.fit(X_esc, Y)
    
    #Dataset with selected features
    model = SelectFromModel(predictor, prefit = True, max_features=features)
    X_new = model.transform(X_esc) 
    
    feature_idx = model.get_support()
    feature_name = X_esc.columns[feature_idx]
    
    X_filtered = pd.DataFrame(X_new, columns=feature_name)
    return X_filtered

In [5]:
X_extracted = feature_extraction(df_gold, 50)
print('Features:', len(X_extracted.columns))

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Features: 543


In [9]:
X_extracted.describe()

,TOPIC0,TOPIC1,TOPIC5,TOPIC7,TOPIC8,TOPIC9,TOPIC10,TOPIC11,TOPIC12,TOPIC13,TOPIC14,TOPIC15,TOPIC16,TOPIC18,TOPIC19,TOPIC20,TOPIC21,TOPIC22,TOPIC23,TOPIC24,TOPIC25,TOPIC27,TOPIC28,TOPIC29,TOPIC30,...,TOPIC1144,TOPIC1145,TOPIC1146,TOPIC1147,TOPIC1150,TOPIC1151,TOPIC1154,TOPIC1157,TOPIC1159,TOPIC1162,TOPIC1164,TOPIC1166,TOPIC1167,TOPIC1174,TOPIC1177,TOPIC1178,TOPIC1184,TOPIC1187,TOPIC1188,TOPIC1189,TOPIC1190,TOPIC1191,TOPIC1194,TOPIC1195,TOPIC1198
count,4.999000e+03,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,...,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000
mean,3.306231e-01,0.038339,0.025644,-0.013265,0.027137,-0.019172,0.006834,-0.000759,0.013968,-0.014278,-0.012270,-0.009661,0.004717,0.003197,-0.001760,0.007420,0.001656,0.008183,-0.002356,-0.009301,-0.004060,-0.006319,-0.005115,0.018857,-0.009639,...,-0.000266,-0.000192,-0.000976,0.000732,-0.000161,-0.000662,-0.000132,0.000702,0.000267,0.000122,-0.000232,-0.000773,-0.000113,-0.000221,0.000334,0.000311,0.000431,0.000341,0.000372,-0.000221,0.000377,0.000394,-0.000252,-0.000593,-0.000026
std,3.383969e-01,0.403167,0.218755,0.215772,0.213750,0.204686,0.191516,0.191247,0.189160,0.185967,0.182857,0.183057,0.175069,0.180327,0.168541,0.172000,0.165829,0.163977,0.161434,0.157109,0.159620,0.152947,0.157833,0.154260,0.154399,...,0.043202,0.043557,0.042862,0.043485,0.042290,0.043082,0.042713,0.042520,0.042810,0.041828,0.042373,0.042134,0.042317,0.041311,0.041503,0.041855,0.041748,0.041996,0.041318,0.041186,0.041642,0.041308,0.041146,0.041485,0.041442
min,3.837932e-09,-1.293594,-1.286837,-3.142296,-1.447764,-1.415833,-1.630015,-1.021141,-1.250596,-1.120656,-0.971834,-1.828448,-0.574876,-2.415860,-1.350506,-1.088589,-1.193066,-0.753826,-1.569247,-1.544528,-1.745882,-0.688386,-1.177491,-0.801055,-1.672140,...,-0.233318,-0.224158,-0.207616,-0.217267,-0.275546,-0.217136,-0.189808,-0.214449,-0.335117,-0.228991,-0.209995,-0.236788,-0.290261,-0.303338,-0.241071,-0.182360,-0.182713,-0.310021,-0.244161,-0.212664,-0.191053,-0.185330,-0.318198,-0.190675,-0.206651
25%,7.756629e-02,0.000225,-0.018752,-0.041362,-0.016837,-0.072323,-0.018669,-0.043648,-0.025011,-0.058783,-0.063942,-0.060020,-0.040587,-0.028025,-0.021934,-0.036510,-0.041172,-0.044575,-0.041110,-0.046662,-0.024912,-0.057811,-0.033389,-0.038175,-0.066537,...,-0.025196,-0.024219,-0.024558,-0.023838,-0.023267,-0.024195,-0.023766,-0.022985,-0.023543,-0.024043,-0.023212,-0.024102,-0.022655,-0.023731,-0.022910,-0.023043,-0.023013,-0.023068,-0.022696,-0.022284,-0.022201,-0.022346,-0.023289,-0.023812,-0.023276
50%,1.649193e-01,0.012539,0.021634,-0.008969,0.016973,-0.011436,0.009309,-0.002406,0.009514,-0.011100,-0.009663,-0.005915,-0.002899,0.008302,0.003752,0.007486,-0.000899,-0.002025,-0.007305,0.001464,0.003935,-0.007679,0.000072,0.012684,-0.014456,...,-0.000660,-0.000476,-0.000910,0.000136,0.000040,-0.000849,-0.000558,0.000927,0.001319,-0.000297,-0.000048,-0.000945,-0.000165,-0.000283,0.000018,0.000711,-0.000084,-0.000334,0.000383,-0.000566,0.000183,-0.000119,0.000045,-0.000547,-0.000407
75%,6.059417e-01,0.045823,0.074149,0.017938,0.057837,0.048935,0.052480,0.029572,0.049953,0.042601,0.030643,0.029704,0.023935,0.049438,0.038362,0.065635,0.038414,0.032791,0.031387,0.051447,0.034714,0.038379,0.037022,0.073952,0.026422,...,0.024190,0.023796,0.023442,0.024513,0.022766,0.023291,0.023186,0.024465,0.024126,0.023368,0.023220,0.022029,0.022864,0.022958,0.023511,0.024222,0.023121,0.023615,0.023889,0.022671,0.023164,0.022610,0.022301,0.023016,0.022572
max,2.031096e+0

In [6]:
df_gold_attr = X_extracted
df_gold_labels = pd.DataFrame(df_gold['class'])
df_gold_labels = df_gold_labels.rename(columns={"class": "y"})

In [7]:
for column in df_gold_labels.columns:
    df_gold_labels[column] = [True if row == 'pos' else False for row in df_gold_labels[column]]

In [8]:
df_gold_attr.to_csv('../Sinteticas/sentiment-polarity/database_attr.csv', sep=',', index=False)
df_gold_labels.to_csv('../Sinteticas/sentiment-polarity/database_labels.csv', sep=',', index=False)